# acq01原始数据特征提取

## 1、原始数据清理

#### 时间清理

## 2、特征提取

#### 原始数据时间筛选

#### 特征提取统计计算 - 改名

In [1]:
# 加载所需库
# 科学计算
import pandas as pd
# 矩阵运算
import numpy as np
# json
import json
# 数据可视化
import seaborn as sns
# 统计函数库
import scipy.stats as st
# 在日期时间库中拿出三个类导入
from datetime import datetime,date,timedelta
# 时间库，基于操作系统层面，datetime是封装time后的
import time
# python内建集合模块，提供许多扩展的类
import collections

In [2]:
# 定义核心事件
# 1.事件对应标识字典
path_dict={ '/regisiter.html':'R', '/card/certification.html':'C', '/card/bankcard.html':'B',
          '/card/handlecard.html':'H', '/card/handlecard2.html':'H', '/card/supermarket.html':'S',
          '/card/addcard.html':'D', '/card/addcreditcard.html': 'A', '/lend/lendInfo.html':'L',
          '/lend/lendbycard.html':'L','/my/myinfo.html':'M','/login.html':'login'}

# 2.事件列表
core_path=['/regisiter.html','/card/certification.html','/card/bankcard.html','/card/handlecard.html',
          '/card/handlecard2.html','/card/supermarket.html','/card/addcard.html','/card/addcreditcard.html',
           '/lend/lendInfo.html','/lend/lendbycard.html','/my/myinfo.html','/login.html']

# 3.标识列表
core_path_ABB=['R','C','B','H','S','D','A','L','M','login']

# 4.应该是筛选条件列表!!!!
# 定义加入计算的appid
jk_appid=['jkwx', 'jkand', 'jkand2', 'jkios', 'yzand1', 'jkiosspeed', 'jkiossolar','jkioslunar',
         'jkmjand_1','jkmjios_1','jkiosxjjsd','jkiosjqb','jkiosxysjd','jkiosjqd','jkiosxejsd','jkiossjdjk',
         'jkiosxjjk','jkiospphd','jkiosjyq','jkiosxedk','jkandjqb','jkandxedk','jkandpphd','jkori','jkoriand2','jkoriios']

In [3]:
#实现什么功能，逻辑
def getTimeInterval(df):
    '''
    输入一个dataframe，计算每条数据的停留时间，即用下一条数据的sessionId当作本次行为的结束时间。
    '''
    df = df.sort_values('sessionId', ascending = 'True')
    k=((np.diff(df.date.values)).astype(int)/1000000000).astype(int)
    k=[x if x<1800 else -1 for x in k] + [0] # 如果停留时间超过30分钟则认定为-1（登出而非停留）
    p=df.path.values
    return p,k



In [4]:
# 生成核心时间的活跃天数+appid的个数以及页面访问的个数

def getCommonSDK(df):
    '''
    特征提取主代码，输入是原始数据，输出是特征。
    
    所有的统计计算都是通过groupby完成的。如df.groupby('ugid')['appid'].nunique() | df.groupby('ugid').appid.nunique()是指把df按照ugid进行groupby，对每个ugid下的
    appid进行去重个数计算(nunique())
    '''
    # 分组,分组后数据的形式是一块一块的
    g=df.groupby('ugid')
    # 按appid去重后各组剩余记录数，计算客户客户端数量
    result=g.appid.nunique().reset_index().rename(columns={'appid':'appid_cnt'})
    # 计算访问页面数量，即事件
    tmp=g.path.nunique().reset_index()
    tmp.columns=['ugid','distinct_event_cnt']
    result=pd.merge(result, tmp,how= 'left', on = 'ugid')  
    
    # 筛选包含核心事件的数据，以下特征只针对核心事件
    # 将事件用标识替换
    df_core=df[df['path'].isin(core_path)]
    df_core['path']=[path_dict[x] for x in df_core.path]
    # 计算各类型核心事件的活跃天数
    for item in core_path_ABB:
        
        # 这里循环的item是核心事件
        temp = df_core[df_core['path']==item].groupby('ugid').day.nunique().reset_index().rename(columns = {'day':item+"_active_days"})
        temp = temp[temp.ugid == temp.ugid] 
        result = pd.merge(result, temp, how='left', on ='ugid')
    
    # inplace=true，在源表中替换空值
    result.fillna(0,inplace=True)
    l=result.shape[1]
    #result[list(result.columns[3:l])]=result[list(result.columns[3:l])].astype(int)
    #result.fillna(0,inplace=True)
    # 将计算后的结果转为整型
    result.iloc[:,1:]=result.iloc[:,1:].astype(int)
    return result

In [5]:
# lambda 匿名函数关键字
# 正序排列sessionId
sortTime= lambda x: x.sort_values('sessionId',ascending=True)
# 是否有访问注册页面
view_register=lambda x:'/regisiter.html' in x
def getDayBaseSDK(df):
    '''
    特征提取代码。提取剩余特征,
    '''
    # 有活动的天数
    result=df.groupby('ugid').day.nunique().reset_index().rename(columns={'day':'active_days'})
    
    # 整合path的顺序,axis=1为删除列时需要加的参数
    df=df.groupby('ugid',sort=False).apply(sortTime).reset_index(0,drop=True).reset_index().drop('index',axis=1)
    def show_path(path_list):
        return path_list.values
    # 拿出访问的页面列表
    tmp=df.groupby('ugid').path.apply(show_path).reset_index().rename(columns={'path':'event_path_sequence'})
    result=pd.merge(result,tmp,how='left',on='ugid')
    
    # 判断有无注册页面
    result['register_YN']=result.event_path_sequence.apply(view_register)
    
    # 计算访问过注册页面的zuid数量（即设备）
    tmp=df[df['path']=='/regisiter.html'].groupby('ugid').zuid.nunique().reset_index().rename(columns={'zuid':'register_device_cnt'})
    result=pd.merge(result,tmp,how='left',on='ugid')
    
    result.fillna(0,inplace=True)
    result['active_days']=result['active_days'].astype(int)
    result['register_device_cnt']=result['register_device_cnt'].astype(int)
    #result['last10events_pattern']=[x[-10:] for x in result.event_path_sequence]
    result=result.drop('event_path_sequence',axis=1)
    return result

In [6]:
def mergeData(scr,tmp):
    data=pd.merge(scr, tmp,how = 'left', on = 'ugid')
    return data

In [7]:
# allfeature=getDayBaseWTData(acq01,30)
def getDayBaseWTData(df,timePeriod):
    '''
    特征提取主入口，输入df为原始数据dataframe，timePeriod为筛选数据的时间窗口，输出为特征dataframe
    '''
    
    # 配置筛选数据时间窗口及前缀http://dict.youdao.com/search?q=getMonthBaseWTData&keyfrom=chrome.extension
    suffix_dict={30:'last1m_',7:'last1w_',1:'last1d_'}
    
    # 筛选时间窗口内的数据（过去一天|过去一周|过去一个月）
    # reset_index(drop=true)新建索引并删除原索引
    data=df[(df.date < df.last_decision_tm) & (df.date > df.last_decision_tm + timedelta(days=-timePeriod))].reset_index(drop=True)
    
    # 调取特征提取函数
    
    # 函数一,计算客户端数，各类型事件活跃天数
    tmp=getCommonSDK(data)
    tmp.columns=list(tmp.columns[:1])+list(suffix_dict[timePeriod] + tmp.columns[1:])
    result=tmp.copy()
    
    # 函数二，计算是否访问过注册页面，访问注册页面的设备数
    tmp=getDayBaseSDK(data)
    tmp.columns=list(tmp.columns[:1])+list(suffix_dict[timePeriod] + tmp.columns[1:])
    result=pd.merge(result,tmp,how='left',on='ugid')
    return result

In [8]:
# tmp1 = getMonthBaseWTData(acq01,"weekday",3)
def getMonthBaseWTData(df,weekday,hourPeriod):
    '''
    筛选时间-判断是否为工作日或休息日，再执行特征提取
    
    '''
    # 筛选时间窗口
    data=df[ (df.date < df.last_decision_tm) & (df.date > df.last_decision_tm + timedelta(days=-30)) ].reset_index(drop=True)

    # 工作日核心事件字段衍生
    if weekday == 'weekday':
        # 筛选工作日
        data=data[data.day.apply(datetime.weekday)<=4 ].reset_index(drop=True)
        
        # 特征流程
        tmp=getCommonSDK(data)
        tmp.columns=list(tmp.columns[:1])+list('last1m_'+tmp.columns[1:]+"_"+weekday)
    # 休息日核心事件字段衍生
    elif weekday == 'weekend':
        # 筛选休息日
        data=data[data.day.apply(datetime.weekday) > 4 ].reset_index(drop=True)
        tmp=getCommonSDK(data)
        tmp.columns=list(tmp.columns[:1])+list('last1m_'+tmp.columns[1:]+"_"+weekday)
    # 0时开始统计每四个小时时间段核心事件字段衍生
    else:
        # 提取24小时
        data['timePiece']=[x.hour for x in data.date]
        # 按照时间筛选timePeriod
        data_tmp=data[(data.timePiece >= 4*hourPeriod) & (data.timePiece < 4+hourPeriod*4)].reset_index(drop=True)
        # 特征提取
        tmp=getCommonSDK(data_tmp)
        tmp.columns=list(tmp.columns[:1])+list('last1m_'+tmp.columns[1:]+"_HourPeriod"+ str(hourPeriod))        
    return tmp

In [9]:
#以上所有函数功能 返回结果，逻辑
#下面代码如何将时间转换的

# reader是个迭代器，不是数据，将迭代器通过pd.concat拼接之后才是最终读取出来的数据
# dtype每列数据的数据类型，iterator逐块处理文件,chunksize文件块的大小
reader = pd.read_csv("./data_mid/20171016_data_acq01.201611_201705.csv",dtype = {'utime':'str','ctime':'str','sessionId':'str'},
                     iterator=True, chunksize=1000000)
#读出数据,ignore_index各文件块数据是否重新赋值索引,返回一个dataframe
acq01 = pd.concat(reader, ignore_index=False)

#对dataframe进行去重，inplace是否在源数据上进行操作
acq01.drop_duplicates(inplace=True)

# 定义函数：将unix时间戳转为一个标准格式的时间，除以1000因为时间戳单位为毫秒,再格式化为'2016-11-30 15:56:56'形式
converDate=lambda x:datetime.fromtimestamp(int(x)/1000).strftime('%Y-%m-%d %H:%M:%S')
#dataframe行和列的元组
acq01.shape

FileNotFoundError: File b'./data_mid/20171122_data_acq01.201706_201708.csv' does not exist

In [17]:
%%time
# 输出代码运行时间,必须放在代码段的第一行
# 异常值处理,将ctime列中null替换为-1并把此列转为浮点型
acq01['ctime'] = acq01['ctime'].replace('null','-1').astype('float')

CPU times: user 13.6 s, sys: 1.28 s, total: 14.8 s
Wall time: 14.8 s


In [18]:
%%time

# 异常值处理
acq01['sessionId'] = acq01['sessionId'].replace('null','-1').astype('float')

CPU times: user 13 s, sys: 1.33 s, total: 14.4 s
Wall time: 14.3 s


In [19]:
# 异常值处理,sessionId缺失的数据采集时间乘以1000
acq01['sessionId'] = acq01['sessionId'].mask(acq01['sessionId']==-1, acq01['ctime']*1000)

# 筛选目标对象，ugid为空的删除
acq01=acq01[(acq01.ugid==acq01.ugid) & (acq01['ugid']!='null')]
# 筛选指定的客户端类型appid
acq01=acq01[acq01.appid.isin(jk_appid)]

In [20]:
%%time

# 特征加工准备、时间清理
# strptime将字符串转为日期
acq01['date']=acq01.sessionId.fillna(0).apply(converDate)
acq01['time']=[x.split()[1] for x in acq01['date']]
acq01['day']=[x.split()[0] for x in acq01['date']]
acq01['day']=[datetime.strptime(x,'%Y-%m-%d') for x in acq01['day']]
acq01['date']=[datetime.strptime(x,'%Y-%m-%d %H:%M:%S') for x in acq01['date']]

CPU times: user 21min 39s, sys: 21.7 s, total: 22min 1s
Wall time: 22min 3s


In [21]:
overdue_status = pd.read_csv('./data_raw/app_20w_merge_jiekuan.csv').drop(['month','target','user_gid','last_decision_dt','SamplingWeight'],axis=1).rename(columns={'create_time':'last_decision_tm'})
#overdue_status = pd.read_csv('./data_raw/app_20w.csv')

In [22]:
print (overdue_status)
print (len(overdue_status))
print (len(acq01))

              last_decision_tm                                  ugid
0       20FEB2017:18:25:03.300  78846e44-16de-47de-9ee4-dffff4e3d726
1       12MAY2017:21:49:18.963  6956744e-ad27-4355-855e-4c1fc3cd04a9
2       10APR2017:10:42:48.293  90a8ddd4-2f9d-4b41-ad98-929448a30735
3       08FEB2017:14:11:00.163  67183c15-0fe4-4e1f-8037-8344dcc97397
4       24APR2017:17:10:31.363  2903bddf-e5df-44f2-b26e-b537c4dc0197
5       18MAY2017:13:20:44.223  c45626ae-539f-4ca1-acb1-8744cc5c99bf
6       18FEB2017:01:55:53.087  a361468d-2ca5-48dd-aa93-6d3f8d802366
7       08JAN2017:10:49:47.533  25970d65-97e6-4aa1-9fc5-f6997c639f99
8       01APR2017:00:39:45.270  e2843946-0588-451b-8570-5395e34159a1
9       24APR2017:18:01:50.103  6fba0f0e-443e-4947-9519-24838e93f31c
10      01FEB2017:11:05:27.973  00043E90-255D-4AE4-9D7C-DEBF10E8E3A4
11      30MAR2017:07:24:38.377  5ffb50c4-9660-4557-888e-10d49f78efbc
12      23FEB2017:08:10:04.313  93f470fa-b844-4d04-9bcf-705a48a24bb4
13      20JAN2017:14:25:15.457  f9

In [23]:
# main_pd = pd.read_csv('./data_raw/20171016_app_20w.csv')
# main_pd = main_pd[['ugid', 'user_gid']]
# # 读取申请数据，取申请时间
# overdue_status = pd.read_csv('./data_raw/decision_tm_r360.csv')
# #只保留两个字段
# overdue_status = overdue_status[['user_gid', 'decision_tm1']].rename(columns={'decision_tm1':'last_decision_tm'})
# overdue_status = pd.merge(main_pd, overdue_status, how='left', on = 'user_gid')
# overdue_status = overdue_status[['ugid', 'last_decision_tm']]

In [23]:
# acq01为主表
acq01 = pd.merge(acq01, overdue_status, how='right', on = 'ugid')

In [24]:
# 删除申请时间缺失的记录
acq01 = acq01[acq01.last_decision_tm == acq01.last_decision_tm]

In [25]:
print(len(acq01))
#print(pd.to_datetime(acq01['last_decision_tm']))

39247686


In [26]:
%%time

# 转换申请时间格式
# 使用正则表达式形式去替换
acq01['last_decision_tm'] = acq01['last_decision_tm'].replace('DEC', '-12-',regex=True).\
replace('NOV', '-11-',regex=True).\
replace('OCT', '-10-',regex=True).\
replace('SEP', '-09-',regex=True).\
replace('AUG', '-08-',regex=True).\
replace('JUL', '-07-',regex=True).\
replace('JUN', '-06-',regex=True).\
replace('MAY', '-05-',regex=True).\
replace('APR', '-04-',regex=True).\
replace('MAR', '-03-',regex=True).\
replace('FEB', '-02-',regex=True).\
replace('JAN', '-01-',regex=True)
# acq01['last_decision_tm'] = acq01['last_decision_tm'].apply(lambda x :datetime.strptime(x, '%Y-%m-%d')) 
acq01['last_decision_tm'] = acq01['last_decision_tm'].apply(lambda x :datetime.strptime(x, '%d-%m-%Y:%H:%M:%S.%f'))
acq01['last_decision_tm'] = pd.to_datetime(acq01['last_decision_tm'])

CPU times: user 16min 29s, sys: 13.6 s, total: 16min 42s
Wall time: 16min 44s


In [27]:
acq01 = acq01[acq01.path == acq01.path ]

In [28]:
acq01

,ugid,zuid,appid,utime,ctime,path,remote_addr,sessionId,date,time,day,last_decision_tm
0,5ffb50c4-9660-4557-888e-10d49f78efbc,88727525f618472da5d9d6ede9b4c923|963693720080768,jkand2,1478140679525,1.478141e+09,/index.html,NaN,1.478141e+12,2016-11-02 19:37:59,19:37:59,2016-11-02,2017-03-30 07:24:38.377
1,5ffb50c4-9660-4557-888e-10d49f78efbc,88727525f618472da5d9d6ede9b4c923|963693720080768,jkand2,1478140693657,1.478141e+09,/v1/index.html,NaN,1.478141e+12,2016-11-02 19:38:13,19:38:13,2016-11-02,2017-03-30 07:24:38.377
2,5ffb50c4-9660-4557-888e-10d49f78efbc,88727525f618472da5d9d6ede9b4c923|963693720080768,jkand2,1478140696243,1.478141e+09,/v1/index.html,NaN,1.478141e+12,2016-11-02 19:38:16,19:38:16,2016-11-02,2017-03-30 07:24:38.377
3,5ffb50c4-9660-4557-888e-10d49f78efbc,ae4d60baa85184a9877c1337b1379441|123456789002,jkand2,1487410502437,1.487411e+09,/index.html,106.226.12.80,1.487411e+12,2017-02-18 02:35:02,02:35:02,2017-02-18,2017-03-30 07:24:38.377
4,5ffb50c4-9660-4557-888e-10d49f78efbc,ae4d60baa85184a9877c1337b1379441|123456789002,jkand2,1487410508384,1.487411e+09,/iridium/html/newlist.html,106.226.12.80,1.487411e+12,2017-02-18 02:35:08,02:35:08,2017-02-18,2017-03-30 07:24:38.377
5,5ffb50c4-9660-4557-888e-10d49f78efbc,ae4d60baa85184a9877c1337b1379441|123456789002,jkand2,1487410513760,1.487411e+09,/iridium/html/xrz.html,106.226.12.80,1.487411e+12,2017-02-18 02:35:13,02:35:13,2017-02-18,2017-03-30 07:24:38.377
6,5ffb50c4-9660-4557-888e-10d49f78efbc,ae4d60baa85184a9877c1337b1379441|123456789002,jkand2,1487410708936,1.487411e+09,/index.html,106.226.12.80,1.487411e+12,2017-02-18 02:38:28,02:38:28,2017-02-18,2017-03-30 07:24:38.377
7,5ffb50c4-9660-4557-888e-10d49f78efbc,e8193a0697899d3ab431bdefa88de8e4|123456789002,jkand2,1487410307044,1.487410e+09,/index.html,106.226.12.80,1.487410e+12,2017-02-18 02:31:47,02:31:47,2017-02-18,2017-03-30 07:24:38.377
8,5ffb50c4-9660-4557-888e-10d49f78efbc,e8193a0697899d3ab431bdefa88de8e4|123456789002,jkand2,1487410313835,1.487410e+09,/my/myinfo.html,106.226.12.80,1.487410e+12,2017-02-18 02:31:53,02:31:53,2017-02-18,2017-03-30 07:24:38.377
9,5ffb50c4-9660-4557-888e-10d49f78efbc,e8193a0697899d3ab431bdefa88de8e4|123456789002,jkand2,1487410318974,1.487410e+09,/iridium/html/newlist.html,106.226.12.80,1.487410e+12,2017-02-18 02:31:58,02:31:58,2017-02-18,2017-03-30 07:24:38.377


# 特征提取入口

* 在时间窗口内用全部数据进行计算
    * 过去三十天访问注册页面次数（eg）
* 在时间窗口内定义不同的时间组别：
    * 休息日和工作日
        * 过去三十天工作日访问注册页面次数（eg）
        * 过去三十天休息日访问注册页面次数（eg）
    * 每天的不同时间段
        * 过去三十天在凌晨0-4点访问注册页面次数（eg）timePeriod0-5
        * 过去三十天在12-16点访问注册页面次数（eg）

In [29]:
%%time
# 提取过去一个月的特征
allfeature=getDayBaseWTData(acq01,30)

# 提取过去一周特征并与过去一个月特征集合并
allfeature=mergeData(allfeature, getDayBaseWTData(acq01,7))

# 提取过去一天特征并与前面所有特征集合并
allfeature=mergeData(allfeature, getDayBaseWTData(acq01,1))

# 填补缺失值
allfeature.fillna(0,inplace=True)

/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


CPU times: user 15min 35s, sys: 50.2 s, total: 16min 25s
Wall time: 16min 24s


In [30]:
%%time

# 区分工作日和休息日的特征
allfeature=mergeData(allfeature, getMonthBaseWTData(acq01,"weekday",3))
allfeature=mergeData(allfeature, getMonthBaseWTData(acq01,"weekend",3))

# 区分每天6个时间段的特征
allfeature=mergeData(allfeature, getMonthBaseWTData(acq01,"noweekday",0))
allfeature=mergeData(allfeature, getMonthBaseWTData(acq01,"noweekday",1))
allfeature=mergeData(allfeature, getMonthBaseWTData(acq01,"noweekday",2))
allfeature=mergeData(allfeature, getMonthBaseWTData(acq01,"noweekday",3))
allfeature=mergeData(allfeature, getMonthBaseWTData(acq01,"noweekday",4))
allfeature=mergeData(allfeature, getMonthBaseWTData(acq01,"noweekday",5))
allfeature.fillna(0,inplace=True)

/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


CPU times: user 9min 6s, sys: 27.3 s, total: 9min 33s
Wall time: 9min 32s


In [31]:
# 一小时内的核心事件字段衍生
data=acq01[(acq01.date < acq01.last_decision_tm) & (acq01.date > acq01.last_decision_tm + timedelta(hours=-1)) ].reset_index(drop=True)
tmp=getCommonSDK(data)
tmp.columns=list(tmp.columns[:1])+list('last1h_' + tmp.columns[1:])

/opt/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [32]:
allfeature=mergeData(allfeature,tmp)

In [33]:
allfeature.fillna(0,inplace=True)

In [34]:
allfeature.to_csv('./data_last/feature_acq01_201611_201705.csv', index = False)

In [35]:
allfeature

,ugid,last1m_appid_cnt,last1m_distinct_event_cnt,last1m_R_active_days,last1m_C_active_days,last1m_B_active_days,last1m_H_active_days,last1m_S_active_days,last1m_D_active_days,last1m_A_active_days,...,last1h_R_active_days,last1h_C_active_days,last1h_B_active_days,last1h_H_active_days,last1h_S_active_days,last1h_D_active_days,last1h_A_active_days,last1h_L_active_days,last1h_M_active_days,last1h_login_active_days
0,00001d09-f989-42b4-9a87-beb964f4b74c,1,10,0,3,2,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0000c8ec-765e-48f7-85a2-c54ffe8c0c39,1,6,0,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,00011a94-448b-4871-b1c6-f16404ad04be,2,7,0,0,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0001363f-1d5d-4a0a-99bf-1f31b5630618,1,21,0,0,1,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,000161b3-41e1-4426-935a-73b253a5af27,2,8,2,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,00021706-7406-42ac-b839-0ac89b17c44b,1,22,0,1,0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,00028ddc-c3ab-4706-bc7c-524359438c9d,1,18,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0002e5d7-3d3f-4a60-bc53-9359c5a025ad,1,25,0,1,1,1,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,00035da5-eb9c-4876-abaf-d2858e5f1153,2,18,0,1,1,0,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,00037e27-a131-4482-a2f2-e6cc59025360,1,11,0,1,0,0,0,0,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
